In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'LoadVariables', 'TransmissionLines', 'GeneratorsVariables'],
        'maps': ['LoadMaps','GeneratorsMaps'],
        'variable2D': ['HourlyVariation'],
        'scalars': ['Scalars']}
db = dbFromWB(os.path.join(d['data'],'E4.xlsx'), read)
readSets(db)

# The ```mBasicPH``` model

In [2]:
m = mBasicPH.mSimple(db)

The class specifies a linear electricity system model with the following main features:
* Electricity and heat markets.
* Hourly variation in generation capacity and demand.
* No flexible demand systems, but load can be shedded if the marginal costs exceed a threshold value.
* Geographically separated markets; electricity markets are linked via transmission lines (HVDC-links) enablin trade.
* Different technologies generate electricity and heat. We split them into the following overall categories:
    * ```standard_E```: Only generates electricity. These are the only plant types used in basic models (```mBasic,mBasicInt,mBasicTrade```).
    * ```standard_H```: Works as ```standard_E``` plants, but generates heat instead of electricity.
    * ```BP```: Back-pressure plants. Generates electricity and heat in a fixed ratio ```E2H```>0.
    * ```HP```: Heat pump. Generates heat, but draws on electricity to do so. Also characterized by fixed ratio ```E2H```<0 for heat pumps.

### **The model**

Formally, the model solves:
<a id='model'>
$$\begin{align} \tag{1}
    \max\mbox{ }\text{Welfare} &= \sum_{h,g}\left(u_E\cdot D_{h,g}^E - \sum_{i\in \lbrace \mathcal{I}^{g,E}, \mathcal{I}^{g,BP}\rbrace}mc_i\cdot E_{i,h} \right) - \left(\sum_{l,h}c_l\cdot x_{l,h}\right) \\
                                &+\sum_{h,g}\left(u_H\cdot D_{h,g}^H - \sum_{i\in\lbrace \mathcal{I}^{H,g}, \mathcal{I}^{g,HP}\rbrace} mc_i\cdot H_{i,h}\right) \\ 
    D_{h,g}^{E}&=\sum_{i\in\lbrace \mathcal{I}^{E,g}, \mathcal{I}^{BP,g}, \mathcal{I}^{HP,g}\rbrace} E_{i,h}+\sum_{l\in S_g}(1-\text{ll})\cdot x_{l',h}-x_{l,h} \\
    D_{h,g}^{H}&=\sum_{i\in\lbrace \mathcal{I}^{H,g}, \mathcal{I}^{BP,g}, \mathcal{I}^{HP,g}\rbrace} H_{i,h} \\
    E_{i,h} &= \text{E2H}_i\cdot H_{i,h}, \qquad \forall i\in\lbrace \mathcal{I}^{BP}, \mathcal{I}^{HP}\rbrace \\
    D_{h,g}^E &\in[0, \text{Load}_{h,g}^E] \\ 
    D_{h,g}^H &\in[0, \text{Load}_{h,g}^H] \\ 
    E_{i,h} & \in[0, q_{i,h}^E], \qquad \forall i \in \mathcal{I}^{E}, \mathcal{I}^{BP} \\
    H_{i,h} & \in[0, q_{i,h}^H], \qquad \forall i \in \mathcal{I}^{H}, \mathcal{I}^{HP} \\ 
    x_{l,h} & \in[0, q_l],
\end{align}$$
    
where we have used notation from earlier model descriptions, and we have used the notation $\mathcal{I}^{BP,g}$ to indicate the set of plants that are of a specific type (BP) and belongs to a specific geographic area (g). A full description of the parameters used throughout here would be quite lengthy, so, instead you may look up notation from other notebooks or the note *"Models for Energy Economics"* sections 3-4.

### Supply with different types of technology:

When we talk about different types of plant, the basic models only included an ```id```. Now, instead, we distinguish between three levels of identification (most-to-least granular details):
* ```id```: Most specific; every plant has a specific id. Most variables are specific to ```id```, including ```FuelMix, GeneratingCap_E,OtherMC,FOM,E2H```.
* ```tech```: Is used when some parameter values are the same for a group of ```ids```. This is for example used for the parameter ```InvestCost```. One example of ```tech``` categories are the three different levels of back-pressure plants in the data: ```BP_Coal, BP_NatGas, BP_BioMass```.
* ```modelTech```: This identification is used to indicate how the plants are *modelled*. For instance, the three different types of back-pressure plants all essentially use the same code in the model - just with different parameter values. Other large categories include ```StandardE, StandardH```. The ```StandardE``` for instance includes offshore wind, onshore wind, photovoltaics, condensation plants using coal, condensation plants using natural gas, and nuclear power plants.

The information on how plants are mapped to ```tech``` and ```modelTech``` categories can be accessed through the index ```id2modelTech2tech```: 

In [3]:
m.db['id2modelTech2tech']

MultiIndex([('g1_BH_BioMass', 'standard_H', 'BH_BioMass'),
            ( 'g1_BH_NatGas', 'standard_H',  'BH_NatGas'),
            ('g1_BP_BioMass',         'BP', 'BP_BioMass'),
            (   'g1_BP_Coal',         'BP',    'BP_Coal'),
            ( 'g1_BP_NatGas',         'BP',  'BP_NatGas'),
            (   'g1_CD_Coal', 'standard_E',    'CD_Coal'),
            ( 'g1_CD_NatGas', 'standard_E',  'CD_NatGas'),
            ('g1_CD_Nuclear', 'standard_E', 'CD_Nuclear'),
            (        'g1_HP',         'HP',         'HP'),
            (        'g1_PV', 'standard_E',         'PV'),
            (        'g1_SH', 'standard_H',         'SH'),
            (        'g1_WS', 'standard_E',         'WS'),
            ('g2_BH_BioMass', 'standard_H', 'BH_BioMass'),
            ( 'g2_BH_NatGas', 'standard_H',  'BH_NatGas'),
            ('g2_BP_BioMass',         'BP', 'BP_BioMass'),
            (   'g2_BP_Coal',         'BP',    'BP_Coal'),
            ( 'g2_BP_NatGas',         'BP',  'BP_NatGas'

An auxiliary function is added ```getTechs(techs, db)``` to create the subsets ($\mathcal{I}^{X}$) used in the formal model in [(1)](#model):

In [4]:
mBasicPH.getTechs('BP',db) # returns all back-pressure plants

Index(['g1_BP_BioMass', 'g1_BP_Coal', 'g1_BP_NatGas', 'g2_BP_BioMass',
       'g2_BP_Coal', 'g2_BP_NatGas'],
      dtype='object', name='id')

In [5]:
mBasicPH.getTechs(['BP','HP'], db) # returns all back-pressure and heat pumps

Index(['g1_BP_BioMass', 'g1_BP_Coal', 'g1_BP_NatGas', 'g1_HP', 'g2_BP_BioMass',
       'g2_BP_Coal', 'g2_BP_NatGas', 'g2_HP'],
      dtype='object', name='id')

## Initialize and solve model

The model is initialized with the database as input using: 

In [6]:
m = mBasicPH.mSimple(db)

The core functions of the model class are similar to those of the basic models (```mBasic, mBasicInt, mBasicTrade```). We focus on the ```self.initBlocks``` method here, as this is the specifying the linear program. As usual, the model is simply compiled and solved using the ```self.solve``` method:

In [7]:
m.solve()

Solution status 0: Optimization terminated successfully.


### 1: Global domains

The ```self.globalDomains``` are used to properly broadcast coefficients to the correct domains (we will see how more specifically below). In this model, the domains are definde as:

<img src="snippets/mAdvPH_globalDomains.png" width="1000" height="400">

#### ```Generation_E```

Let's have a look at the domains for the variable ```Generation_E``` (corresponds to $E_{i,h}$). The global domain for this variable is defined by first establishing the correct plant ids:

The ```subsetIdsTech(x, techs, db)``` method uses the ```getTechs``` method to subset the symbol ```x``` based on the technology types ```techs```. This first part returns an index with ```id2g``` where we only keep plants that produce electricity (```self.modelTech_E```):

In [8]:
mBasicPH.subsetIdsTech(m.db['id2g'], m.modelTech_E, m.db) # Only keep electricity-producing plants in the symbol 'id2g'

MultiIndex([(   'g1_BP_Coal', 'g1'),
            ( 'g1_BP_NatGas', 'g1'),
            ('g1_BP_BioMass', 'g1'),
            (   'g2_BP_Coal', 'g2'),
            ( 'g2_BP_NatGas', 'g2'),
            ('g2_BP_BioMass', 'g2'),
            (   'g1_CD_Coal', 'g1'),
            ( 'g1_CD_NatGas', 'g1'),
            ('g1_CD_Nuclear', 'g1'),
            (   'g2_CD_Coal', 'g2'),
            ( 'g2_CD_NatGas', 'g2'),
            ('g2_CD_Nuclear', 'g2'),
            (        'g1_WS', 'g1'),
            (        'g1_PV', 'g1'),
            (        'g2_WS', 'g2'),
            (        'g2_PV', 'g2'),
            (        'g1_HP', 'g1'),
            (        'g2_HP', 'g2')],
           names=['id', 'g'])

The next part of the global domains for ```Generation_E``` is to repeat this index for all hours ```h```. This is done using the auxiliary function ```cartesianProductIndex```. The result is:

In [9]:
m.globalDomains['Generation_E'] # repeat the symbol 'id2g' for all hours, only keep electricity-producing plants.

MultiIndex([(   'g1_BP_Coal', 'g1', 1),
            (   'g1_BP_Coal', 'g1', 2),
            (   'g1_BP_Coal', 'g1', 3),
            (   'g1_BP_Coal', 'g1', 4),
            ( 'g1_BP_NatGas', 'g1', 1),
            ( 'g1_BP_NatGas', 'g1', 2),
            ( 'g1_BP_NatGas', 'g1', 3),
            ( 'g1_BP_NatGas', 'g1', 4),
            ('g1_BP_BioMass', 'g1', 1),
            ('g1_BP_BioMass', 'g1', 2),
            ('g1_BP_BioMass', 'g1', 3),
            ('g1_BP_BioMass', 'g1', 4),
            (   'g2_BP_Coal', 'g2', 1),
            (   'g2_BP_Coal', 'g2', 2),
            (   'g2_BP_Coal', 'g2', 3),
            (   'g2_BP_Coal', 'g2', 4),
            ( 'g2_BP_NatGas', 'g2', 1),
            ( 'g2_BP_NatGas', 'g2', 2),
            ( 'g2_BP_NatGas', 'g2', 3),
            ( 'g2_BP_NatGas', 'g2', 4),
            ('g2_BP_BioMass', 'g2', 1),
            ('g2_BP_BioMass', 'g2', 2),
            ('g2_BP_BioMass', 'g2', 3),
            ('g2_BP_BioMass', 'g2', 4),
            (   'g1_CD_Coal', 'g1', 1),


#### ```PowerToHeat```

The ```PowerToHeat``` symbol represents the constraint $E_{i,h} = \text{E2H}_i\cdot H_{i,h}$. The domain for this constraint is defined as the combination of (1) Plant ids that are either back-pressure or heat pumps, and (2) hours. Note that we use the aliases ```id_alias, h_alias```, as we have ususally done with constraints. The result is:

In [10]:
m.globalDomains['PowerToHeat']

MultiIndex([('g1_BP_BioMass', 1),
            ('g1_BP_BioMass', 2),
            ('g1_BP_BioMass', 3),
            ('g1_BP_BioMass', 4),
            (   'g1_BP_Coal', 1),
            (   'g1_BP_Coal', 2),
            (   'g1_BP_Coal', 3),
            (   'g1_BP_Coal', 4),
            ( 'g1_BP_NatGas', 1),
            ( 'g1_BP_NatGas', 2),
            ( 'g1_BP_NatGas', 3),
            ( 'g1_BP_NatGas', 4),
            (        'g1_HP', 1),
            (        'g1_HP', 2),
            (        'g1_HP', 3),
            (        'g1_HP', 4),
            ('g2_BP_BioMass', 1),
            ('g2_BP_BioMass', 2),
            ('g2_BP_BioMass', 3),
            ('g2_BP_BioMass', 4),
            (   'g2_BP_Coal', 1),
            (   'g2_BP_Coal', 2),
            (   'g2_BP_Coal', 3),
            (   'g2_BP_Coal', 4),
            ( 'g2_BP_NatGas', 1),
            ( 'g2_BP_NatGas', 2),
            ( 'g2_BP_NatGas', 3),
            ( 'g2_BP_NatGas', 4),
            (        'g2_HP', 1),
            ( 

### 2: Cost coefficients $c$

The cost coefficients are as outlined in [(1)](#model): Costs are proportional to electricity generation for ```standard_E, BP``` plants and proportional to heat generation for ```standard_H, HP``` plant types. The costs of demand and transmission are exactly as in ```mBasicTrade```.

<img src="snippets/mAdvPH_c.png" width="1400" height="500">

### 3: Upper bounds $u$

The upper bounds are imposed on the electricity generation for plant types ```standard_E, BP``` and on heat generation for plant types ```standard_H, HP```. This is ensured by using the ```conditions``` argument. The constraints on demand and transmission are equivalent to ```mBasicTrade``` model.

<img src="snippets/mAdvPH_u.png" width="1400" height="500">

The ```conditions``` argument uses the ```rc_pd``` method to subset the parameter by matching indices. So, in this case, the parameter on ```Generation_E``` is

In [11]:
full = lpCompiler.broadcast(m.hourlyGeneratingCap_E, m.globalDomains['Generation_E'])
full

id             h  g 
g1_BP_BioMass  1  g1      25
               2  g1      25
               3  g1      25
               4  g1      25
g1_BP_Coal     1  g1      25
                        ... 
g2_PV          4  g2       0
g2_WS          1  g2    70.0
               2  g2    20.0
               3  g2    60.0
               4  g2     100
Length: 72, dtype: object

Note the length is 72. The ```getTechs(['standard_E','BP'], self.db)``` returns an index of ```id```s that remove 8 of the entries:

In [12]:
rc_pd(full, mBasicPH.getTechs(['standard_E','BP'], m.db))

id             h  g 
g1_BP_BioMass  1  g1      25
               2  g1      25
               3  g1      25
               4  g1      25
g1_BP_Coal     1  g1      25
                        ... 
g2_PV          4  g2       0
g2_WS          1  g2    70.0
               2  g2    20.0
               3  g2    60.0
               4  g2     100
Name: 0, Length: 64, dtype: object

### 4: Lower bounds $l$

Recall that if no bounds are imposed, the model naturally uses zero as the lower bound. So, the only instance where we need to specify a lower bound, is for the heat pumps generation of electricity (because it is negative). So, for the heat pumps, we set this to $-\infty$ instead:

<img src="snippets/mAdvPH_l.png" width="850" height="200">

### 5: Equality constraints $A_{eq}, b_{eq}$